In [48]:
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
import numpy as np

mousepos = np.array([30, 20])

pos = np.zeros((200,2))
pos[:,0] = np.linspace(-100,100,200)

def display():
    global mousepos
    view.clear()
    n = 300
    s1 = pg.ScatterPlotItem(size=10, pen=pg.mkPen(None), brush=pg.mkBrush(255, 255, 255, 120))
    pos = np.random.normal(size=(n,2), loc=mousepos, scale=10).T
    spots = [{'pos': pos[:,i], 'data': 1} for i in range(n)] + [{'pos': [0,0], 'data': 1}]
    s1.addPoints(spots)
    p = pg.ItemGroup()    
    p.addItem(s1)    
    view.addItem(s1)
    
def relaxed():
    global app    
    display()
    app.processEvents()
    
def mouse(pos):
    global mousepos
    
    pos = view.mapSceneToView(pos)
    mousepos = np.array([pos.x(), pos.y()])

def update():
    global mousepos    
    relaxed()

app = pg.mkQApp()
win = pg.GraphicsLayoutWidget()
win.resize(1800,1800)
win.show()
view = win.addViewBox()
view.setAspectLocked(True)
view.setXRange(-100, 100)
view.setAutoPan(False)

view.scene().sigMouseMoved.connect(mouse)

timer = pg.QtCore.QTimer()
timer.timeout.connect(update)
timer.start(5)



In [20]:
%gui qt


In [3]:
print 2

2


In [4]:
from Tkinter import *
import numpy as np
import scipy.linalg, scipy.stats

class TKanvas(object):
    def __init__(self, draw_fn=None, tick_fn=None, event_fn=None, quit_fn=None, w=400, h=400):
        self.root = Tk()
        self.canvas = Canvas(self.root, background = "black", width=w, height=h)
        self.w, self.h = w, h
        self.cx = w/2
        self.cy=h/2
        self.mouse_x = self.cx
        self.mouse_y = self.cy
        self.canvas.pack(expand=1, fill="both")        
        self.draw_fn = draw_fn
        self.tick_fn = tick_fn
        self.quit_fn = quit_fn
        self.event_fn = event_fn
        self.root.iconify()
        self.root.update()
        self.root.deiconify()
        self.root.wm_title("Canvas view: Press ESC to quit")
        self.root.bind("<Any-KeyPress>", lambda ev: self.event("keypress", ev))
        self.root.bind("<Any-KeyRelease>", lambda ev: self.event("keyrelease", ev))
        self.root.bind("<Escape>", self.quit)
        self.root.protocol("WM_DELETE_WINDOW", lambda :self.quit(None))
        self.root.bind( "<Any-Button>", lambda ev: self.event("mousedown", ev))
        self.root.bind( "<Any-ButtonRelease>", lambda ev: self.event("mouseup", ev))
        self.root.bind( "<Any-Motion>", lambda ev: self.event("mousemotion", ev))        
                        
        self.root.update()
        self.root.after(10, self.update)
        
       
    def quit(self, event):
        print("Exiting...")
        if self.quit_fn is not None:
            self.quit_fn()
        self.root.destroy()
                
    def clear(self):
        self.canvas.delete(ALL)
        
    def rectangle(self, x1, y1, x2, y2, **kw):
        return self.canvas.create_rectangle( x1, y1, x2, y2, **kw)        
    
    def error_ellipse(self, mean, cov, scale=1, **kw):
        r = np.linspace(-np.pi, np.pi, 12)
        cov = scipy.linalg.sqrtm(cov)
        p = np.stack([np.cos(r), -np.sin(r)]).T
        q = np.dot(p,cov)*scale*5 + mean
        self.canvas.create_polygon(*q.ravel(), **kw)
        
        
        
    def square(self, x, y, r, **kw):
        return self.rectangle(x-r, y-r, x+r, y+r, **kw)
        
    def arc(self, x1, y1, x2, y2, **kw):
        return self.canvas.create_rectangle( x1, y1, x2, y2, **kw)        
        
    def line(self, x1, y1, x2, y2, **kw ):
        return self.canvas.create_line( x1, y1, x2, y2, **kw)        
        
    def circle(self, x, y, r, **kw):
        return self.oval(x-r, y-r, x+r, y+r, **kw)
        
    def oval(self, x1, y1, x2, y2, **kw ):        
        return self.canvas.create_oval( x1, y1, x2, y2, **kw)        
        
    def text(self, x1, y1, **kw ):
        return self.canvas.create_text(x,y, **kw)        
        
    def move_rel(self, tagOrId, dx,dy):
        print(tagOrId, dx, dy)
        self.canvas.move( tagOrId, dx, dy)
        
    def delete(self, tagOrId):
        self.canvas.delete(tagOrId)
        
    def event(self, event_type, event):        
        if event_type=="mousemotion":
            # track mouse offset
            dx = self.mouse_x - event.x
            dy = self.mouse_y - event.y            
            event.dx = -dx
            event.dy = -dy            
            
            self.mouse_x = event.x
            self.mouse_y = event.y
            
        if self.event_fn is not None:
            self.event_fn(self, event_type, event)    
    
    def normal(self, mean, cov, ppfs=(0.65, 0.75, 0.85, 0.95), **kw):
        for ppf in reversed(sorted(ppfs)):
            scale = scipy.stats.norm.ppf(ppf)
            self.error_ellipse(mean, cov, scale=scale, **kw)        
        
    def update(self):
        if self.draw_fn is not None:            
            self.draw_fn(self)
        self.root.update_idletasks()        
        if self.tick_fn is not None:            
            self.tick_fn(0.01)
        self.root.after(20, self.update)
            
            
### Demo with mouse crosshairs
def track(src, etype, event): 
    pass        
        
def draw(src):
    src.clear()    
    src.line(0, src.mouse_y, src.w, src.mouse_y, fill="red")        
    src.line(src.mouse_x, 0, src.mouse_x, src.h, fill="red")        
    src.normal(np.array([src.mouse_x, src.mouse_y]), np.eye(2)*30,
               smooth=1, outline="blue", fill='')

    
    
    pass
    
c = TKanvas(draw_fn=draw)        



%gui tk

Exiting...


In [67]:
class Gilbert(object):
    """implementS a Gilbert Markov chain, which can switch between two states. Simulates
    bursty behaviour."""    
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        self.state = 0        
    
    def update(self, dt):
        # account for sampling rate              
        p1 = 1-((1-self.p1)**(dt))
        p2 = 1-((1-self.p2)**(dt))
        
        if self.state==0:            
            if np.random.random()<p1:
                self.state = 1
        if self.state==1:            
            if np.random.random()<p2:
                self.state = 0
        return self.state
                
class NoiseCursor(object):
    def __init__(self, noise=0, dropout=[0, 0.1], jump=0.0):
        self.noise = noise
        self.dropout = dropout
        self.jump = jump
        self.gilbert = Gilbert(dropout[0], dropout[1])
        
    def update(self, x, y):
        x = x + np.random.normal(0,self.noise) 
        
        y = y + np.random.normal(0,self.noise) 
        if np.random.uniform(0,1)<self.jump:
            y += np.random.uniform(-500,500)
            x += np.random.uniform(-500,500)
        cutout = self.gilbert.update(1)
        if cutout:
            x,y = np.nan, np.nan
        return x,y
    
    
class NoiseCursorDemo(object):
    def __init__(self, filter=None, **kwargs):
        self.cursor = NoiseCursor(**kwargs)
        c = TKanvas(draw_fn=self.draw)     
        self.target = np.random.uniform(0,400,(2,))
        self.filter = filter
        self.true_history = []
        self.raw_history = []
        self.filtered_history = []
        
    def draw(self, src):        
        src.clear()    
        x,y = self.target
        src.rectangle(x,y,x+30, y+30, fill="grey", outline="white")
        self.true_history.append((src.mouse_x, src.mouse_y))
        # get noised position
        x,y = self.cursor.update(src.mouse_x, src.mouse_y)
        self.raw_history.append((x,y))
        # apply any filtering
        if self.filter:
            x,y = self.filter(x,y)               
        self.filtered_history.append((x,y))
        if not np.isnan(x):
            src.circle(x,y,5, fill="white")
        
    
    

In [75]:
class FilterNoise(object):
    def __init__(self, alpha):
        self.alpha = alpha
        self.x = 0
        
    def update(self, x ):
        if not np.isnan(x):
            self.x = (1-self.alpha)*x + (self.alpha)*self.x
        else:
            return np.nan
        return self.x
    
class TwoAxisFilter(object):
    def __init__(self, mk_filter):
        self.x_filter = mk_filter()
        self.y_filter = mk_filter()
    def __call__(self, x, y):
        fx = self.x_filter.update(x)
        fy = self.y_filter.update(y)
        return fx, fy
    
    
simple_filter = TwoAxisFilter(lambda: FilterNoise(alpha=0.9))        
n = NoiseCursorDemo(filter=simple_filter, noise=10, dropout=[0.02, 0.1], jump=0.05)        
%gui tk
        

In [4]:
c = canvas.Can()

In [6]:
print(2)

2


In [6]:
%gui tk

In [8]:
def down(x,y,n):
    print(x,y,n)
    
canvas.set_mousedown_handler(down)

(156, 152, 1)
(233, 89, 1)
(213, 76, 1)
(149, 121, 1)
(140, 157, 1)
(358, 3, 1)


In [10]:
    
canvas.set_mouseup_handler(down)

(225, 107, 1)
(225, 107, 1)
(225, 107, 1)
(225, 107, 1)
(117, 106, 1)
(202, 157, 1)
